<a href="https://colab.research.google.com/github/Oguzhandd/Team2_EnhancingPassengerSatisfaction/blob/main/google_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
services = ["Inflight wifi service", "Departure/Arrival time convenient", "Ease of Online booking",
            "Gate location","Food and drink", "Online boarding", "Seat comfort",
            "Inflight entertainment", "On-board service", "Leg room service",
            "Baggage handling", "Checkin service", "Inflight service",
            "Cleanliness"]

In [ ]:
import pandas as pd

# Load data
df1 = pd.read_csv("datasets/train.csv")
df2 = pd.read_csv("datasets/test.csv")
#
# print("Train set:\n")
# print(len(df1))
# print(df1.isnull().sum())
# print("\n")
# print("Test set:\n")
# print(len(df2))
# print(df2.isnull().sum())
# print("\n\n")


# Drop Null Raws
df1 = df1.dropna()
df2 = df2.dropna()


# return cleaned data of train set
def clean_data_train():
    return df1


# return cleaned data of test set
def clean_data_test():
    return df2


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import StratifiedKFold


def forward_feature_selection(classifier):
    sfK = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

    sfs = SFS(
        estimator=classifier,
        n_features_to_select='auto',
        tol=0.001,
        # chosen direction in the article
        direction='forward',
        # chosen scoring in the article
        scoring='roc_auc',
        # cv = 10 = 10-fold cross-validation
        cv=sfK,
    )
    return sfs


In [ ]:
from sklearn.preprocessing import LabelEncoder

train = clean_data_train()
test = clean_data_test()

# For Train set
lencoders = {}
for col in train.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    train[col] = lencoders[col].fit_transform(train[col])

# For Test Set
lencoders_t = {}
for col in test.select_dtypes(include=['object']).columns:
    lencoders_t[col] = LabelEncoder()
    test[col] = lencoders_t[col].fit_transform(test[col])
# Detect outliers
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

# Removal of outliers from dataset
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]


def processed_data_train():
    return train


# return cleaned data of test set
def processed_data_test():
    return test


def split_to_train_test(train, test, features, target):
    train_cleaned = train
    test_cleaned = test

    features = features

    target = target  # Target variable

    X_train = train_cleaned[features]
    y_train = train_cleaned[target].to_numpy()  # Convert Series to NumPy array
    X_test = test_cleaned[features]
    y_test = test_cleaned[target].to_numpy()  # Convert Series to NumPy array

    return X_train, y_train, X_test, y_test


In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = services
x_train, y_train, x_test, y_test = split_to_train_test(clean_data_train(), clean_data_test(), features, 'satisfaction')
random_forest = forward_feature_selection(RandomForestClassifier(n_estimators=10))
random_forest = random_forest.fit(x_train, y_train)

print(random_forest.get_feature_names_out())


output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Ease of Online booking' 'Gate location' 'Online boarding'
 'Baggage handling']

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics


def build_and_evaluate_model(X_train, y_train, X_test, y_test, k):
    # scaler = StandardScaler()
    # X_train_scaled = scaler.fit_transform(X_train)
    # X_test_scaled = scaler.transform(X_test)

    model = forward_feature_selection(KNeighborsClassifier(n_neighbors=k))
    model.fit(X_train, y_train)
    print(model.get_feature_names_out())
    # y_pred = model.predict(X_test_scaled)

    # accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    # print("Accuracy = {}".format(accuracy))
    # print(sklearn.metrics.classification_report(y_test, y_pred, digits=5))

    # return accuracy


X_train, y_train, X_test, y_test = split_to_train_test(clean_data_train(), clean_data_test(), services, 'satisfaction')

k = 10  # Number of neighbors in KNN
build_and_evaluate_model(X_train, y_train, X_test, y_test, k)


output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Gate location' 'Online boarding' 'Inflight entertainment'
 'Leg room service' 'Inflight service']

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

features = services
x_train, y_train, x_test, y_test = split_to_train_test(clean_data_train(), clean_data_test(), features, 'satisfaction')
gb_model = forward_feature_selection(GradientBoostingClassifier(n_estimators=10))
gb_model = gb_model.fit(x_train, y_train)

print(gb_model.get_feature_names_out())

output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Online boarding' 'Leg room service']

In [ ]:
from sklearn.tree import DecisionTreeClassifier

df_train_cleaned = clean_data_train()
df_test_cleaned = clean_data_test()

X_train, y_train, X_test, y_test = split_to_train_test(df_train_cleaned, df_test_cleaned, services, 'satisfaction')

decision_tree = forward_feature_selection(DecisionTreeClassifier())
decision_tree = decision_tree.fit(X_train, y_train)
print(decision_tree.get_feature_names_out())

output: ['Inflight wifi service' 'Departure/Arrival time convenient'
 'Gate location' 'Online boarding' 'Inflight service']